In [312]:
from selenium import webdriver
import time
import re
import pandas as pd
import pickle

## 딱 각잡는데까지의 코드

In [316]:
pw = pickle.load( open('pw.pickle', 'rb') )

In [295]:
url = 'https://www.kci.go.kr/kciportal/main.kci'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(1)
driver.set_window_size(5000, 5000)
driver.find_element_by_css_selector('#loginType3').click()
driver.find_element_by_css_selector('#userID').send_keys('jmanchung')
time.sleep(1)
driver.find_element_by_css_selector('#userPW').send_keys(pw)
driver.find_element_by_css_selector('#mLogin > div > button > img').click()
time.sleep(1)
driver.find_element_by_css_selector('#gnb > li:nth-child(2) > a > img').click()
time.sleep(1)
driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(2) > td:nth-child(1) > select').click()
driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(2) > td:nth-child(1) > select > option:nth-child(5)').click()
driver.find_element_by_css_selector('#AKCFrm > div > div.head > table > tbody > tr:nth-child(2) > td:nth-child(2) > input').send_keys('증권학회지')
driver.find_element_by_css_selector('#AKCFrm > div > div.btn_wrap > div > input[type="image"]').click()
time.sleep(1)
driver.find_element_by_css_selector('#conLeft > div > h4:nth-child(5) > a').click()
for k in range(1,10):
    driver.find_element_by_css_selector('#pubdt{}'.format(k)).click()
driver.find_element_by_css_selector('#faceted_pubdt_list > li.chekReSearch.al_right > a > img').click()
time.sleep(1)

## 논문 긁어오는 코드 시작

In [296]:
information_list = []
references_list = []

for page in range(1,1000):
    try : 
        for i in range(1,60+1):
            references = []
            driver.execute_script('window.scrollTo(100,{});'.format(i*100))
            word = driver.find_element_by_css_selector('#poArtiSearList > table > tbody > tr:nth-child({}) > td:nth-child(2) > p'.format(i)).text
            if ','.join(re.findall('2009|2010|2011|2012|2013|2014|2015|2016|2017|',word)).replace(',','') :
                information_list.append(word)
                driver.find_element_by_css_selector('#poArtiSearList > table > tbody > tr:nth-child({}) > td:nth-child(2) > p > label'.format(i)).click()
                time.sleep(1)
                driver.execute_script('window.scrollTo(100,5000);')
                length_ = int(','.join(re.findall('\d',driver.find_element_by_css_selector('#refeList').text)).replace(',',''))
                driver.find_element_by_css_selector('#refeList > img').click()
                for j in range(1,length_+1):
                    driver.execute_script('window.scrollTo(100,{})'.format(100*j))
                    references.append(driver.find_element_by_css_selector('#printArea > div.articleBody > div:nth-child(3) > div > p:nth-child({})'.format(j)).text)
                references_list.append(','.join(references))
                driver.back()
                time.sleep(1)
        driver.find_element_by_css_selector('#sBody > div.section > div > div > div > a:nth-child(5) > img').click()
        time.sleep(1)
    except Exception as e :
        print(e)
        break

Alert Text: None
Message: unexpected alert open: {Alert text : 예기치 않은 에러가 발생했습니다. 지속적인 발생시 관리자에게 문의하시기 바랍니다.}
  (Session info: chrome=70.0.3538.67)
  (Driver info: chromedriver=2.42.591071 (0b695ff80972cc1a65a5cd643186d2ae582cd4ac),platform=Linux 4.15.0-36-generic x86_64)



# making dataframe

In [301]:
df = pd.DataFrame(columns=['ref','info'])
df['ref'] = references_list
df['info'] = information_list
df.to_csv('til_359.csv',index=False)

In [310]:
df.tail(10)

,ref,info
349,1. [학술지(정기간행물)] - 강효석 / 1990 / 기업공개시 공모주 가격결정에...,의무보호예수제도 변천에 따른 실증분석과 이에 근거한 합리적인 방안 도출: 최대주주를...
350,1. [학술지(정기간행물)] - 강준구 / 2003 / 국내기업 자본조달정책의 효율...,분리형 사모 신주인수권부사채 발행제도의 문제점\n윤평식 | 한국증권학회 | 한국증권...
351,1. [학술지(정기간행물)] - 강경훈 / 2011 / 복수평가제도가 신용등급 쇼핑...,국내 회사채 시장의 등급인플레이션 분석\n한재준 | 강경훈 | 배영수 | 한국증권학...
352,1. [보고서] - 강윤식 / 2013 / 사외이사 인력뱅크의 활용과 의의 : 2-...,국민연금 주주권 행사의 독립성 및 전문성 확보 방안\n강윤식 | 위경우 | 한국증권...
353,1. [학술지(정기간행물)] - 고광수 / 2004 / 투자 주체별 포트폴리오 특성...,상장적격성 실질심사제도와 정보비대칭\n이포상 | 박진우 | 한국증권학회 | 한국증권...
354,1. [학술지(정기간행물)] - 안희준 / 2006 / 투자자별 소유지분과 정보비대...,국고채시장의 시장조성활동이 가격발견기능과 유동성에 미치는 영향\n안희준 | 김학겸 ...
355,1. [학술지(정기간행물)] - 강윤식 / 2012 / 사외이사의 독립성과 기업가치...,대규모기업집단 소속 상장회사에 대한 내부거래 규제의 실효성\n김산월 | 연강흠 | ...
356,1. [학술지(정기간행물)] - 강병호 / 2006 / 투자자 거래활동 유형별 주식...,과세제도 정책 변경으로 인한 프로그램 매매의 영향력 및 투자자 거래행태의 구조적 변...
357,1. [학술지(정기간행물)] - 구본일 / 2008 / 우리나라 기업의 자본구조에 ...,한국 상장기업의 자본구조에 관한 재고찰: 상충관계이론 VS. 자본조달순위이론\n이은...
358,1. [학술지(정기간행물)] - 김세권 / 2011 / 부도위험과 주식수익률 / 한...,신용부도스왑 스프레드에 대한 체계적 위험의 영향\n박윤정 | 김정무 | 한국증권학회...


# more tuning process

In [304]:
testing_ls = [[j for i in [info.split('|') for info in information_list[idx].split('\n') ] for j in i] for idx in range(len(information_list))]
paper_name_list = [testing_ls[idx][0] for idx in range(len(testing_ls))]
name_list = [testing_ls[i][1:idx] for i in range(len(testing_ls)) for idx,val in enumerate(testing_ls[i]) if '증권' in val]
name_list = [','.join(name) for name in name_list]

In [309]:
driver.close()